@register
如果你的Python脚本意外终止，但你仍想执行一些任务来保存你的工作、执行清理或打印消息，那么@register在这种情况下非常方便。

In [ ]:
from atexit import register
import time

@register
def terminate():
    print("Goodbye!")

while True:
    print("Hello")
    time.sleep(1)

@singledispatch
允许函数对不同类型的参数有不同的实现，有点像Java等面向对象语言中的函数重载。

In [ ]:
from functools import singledispatch

@singledispatch
def fun(arg):
    print("Called with a single argument")

@fun.register(int)
def _(arg):
    print("Called with an integer")

@fun.register(list)
def _(arg):
    print("Called with a list")

fun(1)  # Prints "Called with an integer"
fun([1, 2, 3])  # Prints "Called with a list"


@repeat
该装饰器的所用是多次调用被修饰函数。这对于调试、压力测试或自动化多个重复任务非常有用。
跟前面的装饰器不同，@repeat接受一个输入参数，

In [ ]:
from functools import wraps
def repeat(number_of_times):
    def decorate(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(number_of_times):
                func(*args, **kwargs)
        return wrapper
    return decorate


@timeit
该装饰器用来测量函数的执行时间并打印出来。这对调试和监控非常有用。在下面的代码片段中，@timeit装饰器测量process_data函数的执行时间，并以秒为单位打印所用的时间。

In [ ]:
import time
from functools import wraps

def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'{func.__name__} took {end - start:.6f} seconds to complete')
        return result
    return wrapper

@timeit
def process_data():
    time.sleep(1)

process_data()
# process_data took 1.000012 seconds to complete


@retry
当函数遇到异常时，该装饰器会强制函数重试多次。它接受三个参数：重试次数、捕获的异常以及重试之间的间隔时间。

In [ ]:
import random
import time
from functools import wraps

def retry(num_retries, exception_to_check, sleep_time=0):
    """
    遇到异常尝试重新执行装饰器
    """
    def decorate(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for i in range(1, num_retries+1):
                try:
                    return func(*args, **kwargs)
                except exception_to_check as e:
                    print(f"{func.__name__} raised {e.__class__.__name__}. Retrying...")
                    if i < num_retries:
                        time.sleep(sleep_time)
            # 尝试多次后仍不成功则抛出异常
            raise e
        return wrapper
    return decorate

@retry(num_retries=3, exception_to_check=ValueError, sleep_time=1)
def random_value():
    value = random.randint(1, 5)
    if value == 3:
        raise ValueError("Value cannot be 3")
    return value

random_value()
# random_value raised ValueError. Retrying...
# 1

random_value()
# 5


@countcall
@countcall用于统计被修饰函数的调用次数。这里的调用次数会缓存在wraps的count属性中。

In [ ]:
from functools import wraps

def countcall(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        result = func(*args, **kwargs)
        print(f'{func.__name__} has been called {wrapper.count} times')
        return result
    wrapper.count = 0
    return wrapper

@countcall
def process_data():
    pass

process_data()
process_data()
process_data()


 @rate_limited
@rate_limited装饰器会在被修饰函数调用太频繁时，休眠一段时间，从而限制函数的调用速度。这在模拟、爬虫、接口调用防过载等场景下非常有用

In [ ]:
import time
from functools import wraps

def rate_limited(max_per_second):
    min_interval = 1.0 / float(max_per_second)
    def decorate(func):
        last_time_called = [0.0]
        @wraps(func)
        def rate_limited_function(*args, **kargs):
            elapsed = time.perf_counter() - last_time_called[0]
            left_to_wait = min_interval - elapsed
            if left_to_wait > 0:
                time.sleep(left_to_wait)
            ret = func(*args, **kargs)
            last_time_called[0] = time.perf_counter()
            return ret
        return rate_limited_function
    return decorate


@dataclass
Python 3.7 引入了@dataclass装饰器，将其加入到标准库，用于装饰类。它主要用于存储数据的类自动生成诸如__init__， __repr__， __eq__， __lt__，__str__ 等特殊函数。这样可以减少模板代码，并使类更加可读和可维护。

另外，@dataclass还提供了现成的美化方法，可以清晰地表示对象，将其转换为JSON格式，等等

In [ ]:
from dataclasses import dataclass, asdict

@dataclass
class Person:
    first_name: str
    last_name: str
    age: int
    job: str

    def __eq__(self, other):
        if isinstance(other, Person):
            return self.age == other.age
        return NotImplemented

    def __lt__(self, other):
        if isinstance(other, Person):
            return self.age < other.age
        return NotImplemented


john = Person(first_name="John", 
              last_name="Doe", 
              age=30, 
              job="doctor",)

anne = Person(first_name="Anne", 
              last_name="Smith", 
              age=40, 
              job="software engineer",)

print(john == anne)
# False

print(anne > john)
# True

asdict(anne)
#{'first_name': 'Anne',
# 'last_name': 'Smith',
# 'age': 40,
# 'job': 'software engineer'}


异步调用

In [ ]:
import threading
def threaded(func):
    def wrapper(*args, **kwargs):
        """Multi-threads a given function based on 'threaded' kwarg and returns the thread or function result."""
        if kwargs.pop("threaded", True):  # run in thread
            thread = threading.Thread(target=func, args=args, kwargs=kwargs, daemon=True)
            thread.start()
            return thread
        else:
            return func(*args, **kwargs)

    return wrapper

线程锁

In [ ]:
class ThreadingLocked:
    def __init__(self):
        """Initializes the decorator class for thread-safe execution of a function or method."""
        self.lock = threading.Lock()

    def __call__(self, f):
        """Run thread-safe execution of function or method."""
        from functools import wraps

        @wraps(f)
        def decorated(*args, **kwargs):
            """Applies thread-safety to the decorated function or method."""
            with self.lock:
                return f(*args, **kwargs)

        return decorated

click
beautiful command line interfaces

In [ ]:
import click

@click.command()
@click.option("--count", default=1, help="Number of greetings.")
@click.option("--name", prompt="Your name", help="The person to greet.")
def hello(count, name):
    """Simple program that greets NAME for a total of COUNT times."""
    for _ in range(count):
        click.echo(f"Hello, {name}!")

if __name__ == '__main__':
    hello()